In [1]:
import arrow
import datetime
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import os
import pandas as pd
import pickle
import xarray as xr

%matplotlib inline

## The Grid ##

In [2]:
mesh = nc.Dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')
#mesh = nc.Dataset('../../../myResults/mesh_mask201702.nc')
gdepw = mesh.variables['gdepw_1d'][0]
bathy = nc.Dataset('/home/sallen/MEOPAR/grid/bathymetry_201702.nc')
#bathy = nc.Dataset('../../../myResults/bathymetry_201702.nc')
lats = bathy.variables['nav_lat'][:]
lons = bathy.variables['nav_lon'][:]
mesh.close()
bathy.close()

## Functions ##

In [3]:
def calculate_transport(basedir, dir1, section):
    filename = 'ariane_positions_quantitative.nc'
    with nc.Dataset(os.path.join('/data/sallen/results/Ariane',basedir, dir1, filename)) as dataset:
#    with nc.Dataset(os.path.join('../',basedir, dir1, filename)) as dataset:
        final_section = dataset.variables['final_section'][:]
        final_transport = dataset.variables['final_transp'][:]
        vic_transport = np.sum(np.where(final_section == section, final_transport, 0))
    return (vic_transport/24.)

In [4]:
def calculate_all(basedir, dir1, section, allv):
    final = {}
    mean = {}    
    filename = 'ariane_positions_quantitative.nc'
    with nc.Dataset(os.path.join('/data/sallen/results/Ariane',basedir, dir1, filename)) as dataset:
#    with nc.Dataset(os.path.join('../',basedir, dir1, filename)) as dataset:
        final_section = dataset.variables['final_section'][:]
        final_transport = dataset.variables['final_transp'][:]
        vic_transport = np.sum(np.where(final_section == section, final_transport, 0))
        for variable in allv:
            if variable == 'final_age':
                scale = 86400. # convert from seconds to days
            else:
                scale = 1.
            final[variable] = dataset.variables[variable][:]/scale
            mean[variable] = np.sum(np.where(final_section == section, final_transport*final[variable], 0))/vic_transport
    return vic_transport/24., mean

In [5]:
def get_data(start, endtime, basedir, section):
    transport = np.zeros(365*2+1)
    timerange = arrow.Arrow.range('day', start, endtime)
    time = []
    for i, r in enumerate(timerange):
        dir1 = r.format('DDMMMYY').lower()
        transport[i] = calculate_transport(basedir, dir1, section)
#        transport[i] = calculate_transport('./', './', section)
        time.append(r.datetime)
    return time, transport[:len(timerange)]  

In [9]:
def get_data_all(start, endtime, basedir, section, allv):
    timerange = arrow.Arrow.range('day', start, endtime)
    length = len(timerange)
    transport = np.zeros(length)
    mean = {}
    for variable in allv:
        mean[variable] = np.zeros(length)
    time = []
    for i, r in enumerate(timerange):
        dir1 = r.format('DDMMMYY').lower()
#        print (dir1)
        transport[i], meanday = calculate_all(basedir, dir1, section, allv)
        for variable in allv:
            mean[variable][i] = meanday[variable]
        time.append(r.datetime)
    return time, transport, mean

In [7]:
def save_position(basedir, dir1, section, fi, ff):
    print(fi, ff)
    filename = 'ariane_positions_quantitative.nc'
    with nc.Dataset(os.path.join('/data/sallen/results/Ariane',basedir, dir1, filename)) as dataset:
#    with nc.Dataset(os.path.join('../',basedir, dir1, filename)) as dataset:
        final_section = dataset.variables['final_section'][:]
        final_transport = dataset.variables['final_transp'][:]
        final_depth = dataset.variables['final_depth'][:]
        final_lon = dataset.variables['final_lon'][:]
        final_lat = dataset.variables['final_lat'][:]
        init_depth = dataset.variables['init_depth'][:]
        init_lat = dataset.variables['init_lat'][:]
        init_lon = dataset.variables['init_lon'][:]
        for part in range(final_section.shape[0]):
            if final_section[part] == section:
                fi.write('{0:4f} {1:4f} {2:4f} {3:4f} \n'.format(
                    init_depth[part], init_lat[part], init_lon[part], final_transport[part]))
                ff.write('{0:4f} {1:4f} {2:4f} {3:4f} \n'.format(
                    final_depth[part], final_lat[part], final_lon[part], final_transport[part]))
    

## Get the Data ##

In [8]:
# Full South Vic -> PR (2)
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'FullSouth', 2, allv)
fullsouth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullsouth_all = fullsouth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullsouth_all.rename(columns={'mycol':variable}, inplace=True)
fullsouth_all
fullsouth_all.to_csv('Vic_to_PR_2015_v3.csv')

01jan15
02jan15
03jan15
04jan15
05jan15
06jan15
07jan15
08jan15
09jan15
10jan15
11jan15
12jan15
13jan15
14jan15
15jan15
16jan15
17jan15
18jan15
19jan15
20jan15
21jan15
22jan15
23jan15
24jan15
25jan15
26jan15
27jan15
28jan15
29jan15
30jan15
31jan15
01feb15
02feb15
03feb15
04feb15
05feb15
06feb15
07feb15
08feb15
09feb15
10feb15
11feb15
12feb15
13feb15
14feb15
15feb15
16feb15
17feb15
18feb15
19feb15
20feb15
21feb15
22feb15
23feb15
24feb15
25feb15
26feb15
27feb15
28feb15
01mar15
02mar15
03mar15
04mar15
05mar15
06mar15
07mar15
08mar15
09mar15
10mar15
11mar15
12mar15
13mar15
14mar15
15mar15
16mar15
17mar15
18mar15
19mar15
20mar15
21mar15
22mar15
23mar15
24mar15
25mar15
26mar15
27mar15
28mar15
29mar15
30mar15
31mar15
01apr15
02apr15
03apr15
04apr15
05apr15
06apr15
07apr15
08apr15
09apr15
10apr15
11apr15
12apr15
13apr15
14apr15
15apr15
16apr15
17apr15
18apr15
19apr15
20apr15
21apr15
22apr15
23apr15
24apr15
25apr15
26apr15
27apr15
28apr15
29apr15
30apr15
01may15
02may15
03may15
04may15
05may15


In [10]:
# Full South Vic -> GI (3)
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'FullSouth', 3, allv)
fullsouth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullsouth_all = fullsouth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullsouth_all.rename(columns={'mycol':variable}, inplace=True)
fullsouth_all
fullsouth_all.to_csv('Vic_to_GI_2015_v3.csv')

In [12]:
# Back North into PR from Vic
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'BackNorth', 2, allv)
fullsouth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullsouth_all = fullsouth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullsouth_all.rename(columns={'mycol':variable}, inplace=True)
fullsouth_all
fullsouth_all.to_csv('PR_from_Vic_2015_v3.csv')

In [11]:
# In GIslands into GI from Vic
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'InGIslands', 2, allv)
fullsouth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullsouth_all = fullsouth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullsouth_all.rename(columns={'mycol':variable}, inplace=True)
fullsouth_all
fullsouth_all.to_csv('GI_from_Vic_2015_v3.csv')

In [13]:
# Full North PR to Vic
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'FullNorth', 2, allv)
fullnorth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullnorth_all = fullnorth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullnorth_all.rename(columns={'mycol':variable}, inplace=True)
fullnorth_all
fullnorth_all.to_csv('PR_to_Vic_2015_v3.csv')

In [14]:
# South G Islands GI to Vic
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'SouthGIslands', 2, allv)
fullsouth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullsouth_all = fullsouth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullsouth_all.rename(columns={'mycol':variable}, inplace=True)
fullsouth_all
fullsouth_all.to_csv('GI_to_Vic_2015_v3.csv')

In [15]:
# BackSouth Vic from PR
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'BackSouth', 2, allv)
fullsouth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullsouth_all = fullsouth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullsouth_all.rename(columns={'mycol':variable}, inplace=True)
fullsouth_all
fullsouth_all.to_csv('Vic_from_PR_2015_v3.csv')

In [16]:
# BackSouth Vic from GI
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']
start = datetime.datetime(2015, 1, 1)
endtime = datetime.datetime(2015, 12, 31)
time, transport, mean = get_data_all(start, endtime, 'BackSouth', 3, allv)
fullnorth_all = pd.DataFrame(data=transport, index=time, columns=['transport'])
for variable in allv:
    fullnorth_all = fullnorth_all.assign(mycol=pd.Series(mean[variable]).values)
    fullnorth_all.rename(columns={'mycol':variable}, inplace=True)
fullnorth_all
fullnorth_all.to_csv('Vic_from_GI_2015_v3.csv')

In [ ]:
fi = open('south_init.txt', 'a')
ff = open('south_final.txt', 'a')
save_position('./', './', 2, fi, ff)
fi.close() 
ff.close()

In [ ]:
#Victoria_sill_j = 178-1;
#    Victoria_sill_i = np.arange(235,302+1)-1
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
with open('south_init.txt', 'r') as fi:
    mydata = np.loadtxt(fi)
my_lats_edges = lats[235-1:302-1, 178-1] - (lats[235, 178] - lats[234, 178])/2.
my_deps_edges = -gdepw[28::-1]
H, xs, ys = np.histogram2d(mydata[:, 0], mydata[:, 1],  bins = [my_deps_edges, my_lats_edges],
                        weights=mydata2[:,3])
mesh = ax[0].pcolormesh(my_lats_edges[:-1], my_deps_edges[:-1], H)
fig.colorbar(mesh, ax=ax[0])

with open('south_final.txt', 'r') as ff:
    mydata2 = np.loadtxt(ff)
my_lons_edges = lons[387-1, 262-1:309-1] - (lons[387, 300] - lons[387, 301])/2.
H, xs, ys = np.histogram2d(mydata2[:, 0], mydata2[:, 2],  bins = [my_deps_edges, my_lons_edges],
                        weights=mydata2[:,3])
mesh = ax[1].pcolormesh(my_lons_edges[:-1], my_deps_edges[:-1], H)
fig.colorbar(mesh, ax=ax[1])


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 6))
ax2 = ax.twinx()
fullsouth_transport.plot(ax=ax, y='transport', label="South to North", legend=False)
fullnorth_transport.plot(ax=ax, y='transport', label="North to South (shallow)", legend=False)
fullsouth_transport_16.plot(ax=ax, y='transport', label="South to North (deep)", legend=False, c='b')
fullnorth_transport_16.plot(ax=ax, y='transport', label="North to South (shallow)", legend=False, c='g')
#low_pass_tide.plot(ax=ax2, c='r')
ax.set_xlim(fullsouth_transport.index[0], time[-1])
ax.legend()

## Calculation of Tides moved to "Calculate Tides" ##
Here just read them

In [ ]:
low_pass_tide = pd.read_csv('low_pass_tide.csv')
low_pass_tide.plot()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = 0
ii, jj = 0-min(0,lag), 290-min(0,lag)
ax.plot(range(ii, jj), fullnorth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = 0
ii, jj = 150-min(0,lag), 290-min(0,lag)
ax.plot(range(ii, jj), fullnorth_transport['transport'][ii+lag:jj+lag], 'o-')
ax2.plot(range(ii, jj), low_pass_tide['uVelocity'][ii:jj], 'r')

## Summer Deep Water Variability ##

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = -3
ii, jj = 190, 300
ax.plot(range(ii, jj),fullsouth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

## Spring Deep Water Variability ##

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = -3
ii, jj = 100, 190
ax.plot(range(ii, jj),fullsouth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

## Winter Deep Water Variability : note change in lag

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax2 = ax.twinx()
lag = -5
ii, jj = 0-lag, 100
ax.plot(range(ii, jj),fullsouth_transport['transport'][ii+lag:jj+lag])
ax2.plot(range(ii, jj), -low_pass_tide['uVelocity'][ii:jj], 'r')

In [ ]:
mesh = nc.Dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')

In [ ]:
tracers = nc.Dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DTracerFields1hV17-02')
iY = 258; iX = 178; iZ = 28
iY2 = 388; iX2 = 271
salinity = tracers.variables['salinity'][0:24*365:24, :iZ, iY, iX]
salinity2 = tracers.variables['salinity'][0:24*365:24, :iZ, iY2, iX2]

In [ ]:
e3t = mesh.variables['e3t_0'][0, :iZ, iY, iX]

In [ ]:
saldiff = np.zeros(365)
for i in range(365):
    saldiff[i] = np.sum(salinity[i]*e3t) - np.sum(salinity2[i]*e3t)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 5))
ax2 = ax.twinx()
ax.plot(saldiff)
ax2.plot(range(0, 290), fullsouth_transport['transport'][0:290], 'g')

In [ ]:
fs = open('gulfisland_fullsouth_early2015', 'wb')
pickle.dump(gulfislands, fs)
fs.close()

In [ ]:
fs = open('fullsouth_early2015', 'wb')
pickle.dump(fullsouth_transport, fs)
fs.close()

In [ ]:
vic_transport_2015 = pickle.load('vic_transport_2015')
plt.plot(vic_transport_2015)

In [ ]:
fs = open('vic_transport_2015', 'wb')
pickle.dump(vic_transport, fs)

In [ ]:
vsq = pd.read_csv('../day_avg_tide_pd.csv')

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(18, 5))
for i, lag in enumerate(range(-7, 7, 3)):
    ax[i].plot(vsq[99+lag:99+365+lag].uVelocity, vic_transport, '*')
#    print (np.correlate(vsq[99+lag:99+365+lag], vic_transport[:]), lag, mode='valid')

In [ ]:
print (final_section[:10])

In [ ]:
print (final_transport[:10])